# Model splitting for parallel and serial MODFLOW 6

The model splitting functionality for MODFLOW 6 is shown in this notebook. Model splitting via the `Mf6Splitter()` class can be performed on groundwater flow models as well as combined groundwater flow and transport models. The `Mf6Splitter()` class maps a model's connectivity and then builds new models, with exchanges and movers between the new models, based on a user defined array of model numbers.

The `Mf6Splitter()` class supports Structured, Vertex, and Unstructured Grid models.

In [ ]:
import flopy
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from flopy.mf6.utils import Mf6Splitter
from flopy.plot import styles

## Example 1: splitting a simple structured grid model

This example shows the basics of using the `Mf6Splitter()` class and applies the method to the Freyberg (1988) model.

In [ ]:
simulation_ws = Path("../data/mf6-freyberg")

Load the simulation

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=simulation_ws)

Change the working directory, write input files, and run the simulation

In [ ]:
# live coding
            

Visualize the head results and boundary conditions from this model.

In [ ]:
# live coding get model and heads


In [ ]:
fig, ax = plt.subplots(figsize=(5, 7))
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
heads = gwf.output.head().get_alldata()[-1]
heads = np.where(heads == 1e30, np.nan, heads)
vmin = np.nanmin(heads)
vmax = np.nanmax(heads)
pc = pmv.plot_array(heads, vmin=vmin, vmax=vmax)
pmv.plot_bc("WEL")
pmv.plot_bc("RIV", color="c")
pmv.plot_bc("CHD")
pmv.plot_grid()
pmv.plot_ibound()
plt.colorbar(pc);

### Creating an array that defines the new models

In order to split models, the model domain must be discretized using unique model numbers. Any number of models can be created, however all of the cells within each model must be contiguous.

The `Mf6Splitter()` class accept arrays that are equal in size to the number of cells per layer (`StructuredGrid` and `VertexGrid`) or the number of model nodes (`UnstructuredGrid`).

In this example, the model is split diagonally into two model domains.

In [ ]:
modelgrid = gwf.modelgrid

In [ ]:
array = np.ones((modelgrid.nrow, modelgrid.ncol), dtype=int)
ncol = 1
for row in range(modelgrid.nrow):
    if row != 0 and row % 2 == 0:
        ncol += 1
    array[row, ncol:] = 2

Plot the two domains that the model will be split into

In [ ]:
fig, ax = plt.subplots(figsize=(5, 7))
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
pc = pmv.plot_array(array)
lc = pmv.plot_grid()
plt.colorbar(pc)
plt.show()

### Splitting the model using `Mf6Splitter()`

The `Mf6Splitter()` class accepts one required parameter and one optional parameter. These parameters are:
   - `sim`: A flopy.mf6.MFSimulation object
   - `modelname`: optional, the name of the model being split. If omitted Mf6Splitter grabs the first groundwater flow model listed in the simulation

In [ ]:
# live coding


The model splitting is then performed by calling the `split_model()` function. `split_model()` accepts an array that is either the same size as the number of cells per layer (`StructuredGrid` and `VertexGrid`) model or the number of nodes in the model (`UnstructuredGrid`).

This function returns a new `MFSimulation` object that contains the split models and exchanges between them

In [ ]:
# live coding create and split simulation


In [ ]:
# now to write and run the simulation


### Visualize and reassemble model output

Both models are visualized side by side

In [ ]:
# visualizing both models side by side
ml0 = new_sim.get_model("freyberg_1")
ml1 = new_sim.get_model("freyberg_2")

In [ ]:
heads0 = ml0.output.head().get_alldata()[-1]
heads1 = ml1.output.head().get_alldata()[-1]

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 7))
pmv = flopy.plot.PlotMapView(ml0, ax=ax0)
pmv.plot_array(heads0, vmin=vmin, vmax=vmax)
pmv.plot_ibound()
pmv.plot_grid()
pmv.plot_bc("WEL")
pmv.plot_bc("RIV", color="c")
pmv.plot_bc("CHD")
ax0.set_title("Model 0")

pmv = flopy.plot.PlotMapView(ml1, ax=ax1)
pc = pmv.plot_array(heads1, vmin=vmin, vmax=vmax)
pmv.plot_ibound()
pmv.plot_bc("WEL")
pmv.plot_bc("RIV", color="c")
pmv.plot_grid()
ax1.set_title("Model 1")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label="Hydraulic heads")

## Example 2: Create a load balanced splitting mask for a model

In the previous examples, the watershed model splitting mask was defined by the user. `Mf6Splitter` also has a method called `optimize_splitting_mask` that creates a mask based on the number of models the user would like to generate.

The `optimize_splitting_mask()` method generates a vertex weighted adjacency graph, based on the number active and inactive nodes in all layers of the model. This adjacency graph is then provided to `pymetis` which does the work for us and returns a membership array for each node.

The `optimize_splitting_mask()` method just needs the number of models supplied to it.

In [ ]:
# live coding


Plot the load balanced array

In [ ]:
# live coding


split, write, and run the split simulation

In [ ]:
sim_ws = Path("../data/load_balanced_split")
new_sim = mfsplit.split_model(array)
new_sim.set_sim_path(sim_ws)
new_sim.write_simulation()
new_sim.run_simulation()

## Saving node mapping to file

`Mf6Splitter` has a method, `save_node_mapping()` to save the internal model splitter's node mapping information to file.

The `save_node_mapping()` method accepts a JSON file name.

In [ ]:
# live code 

## Loading a saved node map from file

`Mf6Splitter` has a `load_node_mapping()` function that allows the user to load an existing node mapping for array reconstuction.

The `load_node_mapping()` function needs the split simulation object and the JSON node mapping file

In [ ]:
new_sim2 = flopy.mf6.MFSimulation.load(sim_ws=sim_ws)

# live code

Plot up the split model's heads

In [ ]:
head_dict = {}
for ix, mname in enumerate(new_sim2.model_names):
    ml = new_sim2.get_model(mname)
    head_dict[ix] = ml.output.head().get_alldata()[-1]

ra_heads = mfsplit.reconstruct_array(head_dict)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 8))

pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax)
pc = pmv.plot_array(ra_heads)
ib = pmv.plot_inactive()
plt.colorbar(pc, shrink=0.8);

More information about the model splitter can be found [here](https://flopy.readthedocs.io/en/latest/Notebooks/mf6_parallel_model_splitting_example.html)